In [25]:
#Importing Libraries

import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import re
from tqdm import tqdm
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.nmf import Nmf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pratyaksh_agarwal1\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pratyaksh_agarwal1\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
#Access data into a dataframe from nested json

f = open('client_data.json')
dt = json.load(f)
df1 = pd.json_normalize(dt)

In [4]:
df = df1[['_id','_source.issue','_source.sub_product','_source.complaint_what_happened','_source.sub_issue']]

In [5]:
df.rename(columns = {'_id':'id'}, inplace = True)
df.rename(columns = {'_source.issue':'issue'}, inplace = True)
df.rename(columns = {'_source.sub_product':'sub_product'}, inplace = True)
df.rename(columns = {'_source.complaint_what_happened':'complaint'}, inplace = True)
df.rename(columns = {'_source.sub_issue':'sub_issue'}, inplace = True)

C:\Users\pratyaksh_agarwal1\Anaconda33\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
def preprocess(text):
    text = word_tokenize(text)
    text = [each.lower() for each in text]
    text = [re.sub('[0-9]+', '', each) for each in text]    
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [re.sub('[xx]+', '', each) for each in text]   
    text = [each for each in text if len(each) > 1]
    text = [each for each in text if ' ' not in each]
    return text

In [7]:
for i in tqdm(range(len(df['complaint']))):
    if len(df['complaint'][i]) != 0:
        df['complaint'][i] = preprocess(df['complaint'][i])

  0%|                                                                                        | 0/78313 [00:00<?, ?it/s]C:\Users\PRATYA~1\AppData\Local\Temp/ipykernel_30404/3611066106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['complaint'][i] = preprocess(df['complaint'][i])
100%|██████████████████████████████████████████████████████████████████████████| 78313/78313 [01:01<00:00, 1266.58it/s]


In [8]:
prev_column = df1['_source.complaint_what_happened']

In [9]:
main_df = df.join(prev_column)

In [10]:
main_df.rename(columns = {'_source.complaint_what_happened':'original_complaint'}, inplace = True)

In [11]:
main_df

,id,issue,sub_product,complaint,sub_issue,original_complaint
0,3211475,Attempts to collect debt not owed,Credit card debt,,Debt is not yours,
1,3229299,Written notification about debt,Credit card debt,"[good, morning, my, name, is, and, appreciate,...",Didn't receive enough information to verify debt,Good morning my name is XXXX XXXX and I apprec...
2,3199379,"Other features, terms, or problems",General-purpose credit card or charge card,"[upgraded, my, card, in, //, and, wa, told, by...",Problem with rewards from credit card,I upgraded my XXXX XXXX card in XX/XX/2018 and...
3,2673060,Trouble during payment process,Conventional home mortgage,,None,
4,3203545,Fees or interest,General-purpose credit card or charge card,,Charged too much interest,
...,...,...,...,...,...,...
78308,3105617,Closing an account,Checking account,,Funds not received from closed account,
78309,3091984,"Other features, terms, or problems",General-purpose credit card or charge card,"[on, wednesday, //, called, chas, my, visa, cr...",Other problem,"On Wednesday, XX/XX/XXXX I called Chas, my XXX..."
78310,3133355,Problem with a lender or other company chargin...,Checking account,"[am, not, familiar, with, pay, and, did, not, ...",Transaction was not authorized,I am not familiar with XXXX pay and did not un...
78311,3110963,"Other features, terms, or problems",General-purpose credit card or charge card,"[have, had, flawless, credit, for, yr, 've, ha...",Problem with balance transfer,I have had flawless credit for 30 yrs. I've ha...


In [12]:
tfidf_vectorizer = TfidfVectorizer(
    min_df=3,
    max_df=0.85,
    max_features=7500,
    ngram_range=(1,2),
    preprocessor=' '.join,
    stop_words = 'english'
)
features = tfidf_vectorizer.fit_transform(df['complaint'])

In [13]:
nmf = NMF(n_components=5,init='nndsvd').fit(features)
nmf_features = nmf.transform(features)

In [14]:
nmf_score = pd.DataFrame(nmf_features)

In [15]:
nmf_score

,0,1,2,3,4
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.001729,0.006257,0.042629,0.021504,0.000907
2,0.033518,0.006546,0.000000,0.004236,0.002315
3,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
78308,0.000000,0.000000,0.000000,0.000000,0.000000
78309,0.064694,0.018198,0.003656,0.000000,0.000000
78310,0.016851,0.012597,0.036830,0.016744,0.000000
78311,0.020917,0.046756,0.010238,0.000000,0.082879


In [16]:
components_df = pd.DataFrame(nmf.components_, columns = tfidf_vectorizer.get_feature_names())

In [32]:
components_df

,aarp,ability,ability pay,able,able access,able close,able help,able make,able open,able pay,...,yearly,yes,yes wa,yesterday,york,young,yr,zero,zip,zip code
0,0.000000,0.003266,0.000000,0.180380,0.005668,0.001378,0.011150,0.002123,0.000000,0.003138,...,0.002064,0.048032,0.009182,0.029994,0.012811,0.012847,0.000210,0.017803,0.004868,0.002770
1,0.011114,0.019417,0.001923,0.007747,0.000000,0.000000,0.000000,0.000000,0.000000,0.002322,...,0.008921,0.000000,0.000000,0.004814,0.000279,0.002796,0.000000,0.016527,0.002670,0.002667
2,0.001573,0.023793,0.005334,0.066890,0.000000,0.004521,0.002525,0.004293,0.000000,0.006820,...,0.000873,0.014266,0.000000,0.010187,0.011408,0.000000,0.032139,0.004021,0.001129,0.001120
3,0.001372,0.014120,0.000000,0.096007,0.011650,0.003801,0.006325,0.003183,0.026247,0.002382,...,0.001572,0.009975,0.000254,0.021425,0.010901,0.004446,0.000000,0.008227,0.000000,0.000000
4,0.000000,0.021248,0.005006,0.045496,0.000000,0.000000,0.001364,0.020047,0.000000,0.018040,...,0.002911,0.000761,0.000471,0.004221,0.000000,0.000779,0.010826,0.011696,0.004151,0.004649


In [17]:
for topic in range(components_df.shape[0]):

    tmp = components_df.iloc[topic]

    print(f'For topic {topic+1} the words with the highest value are:')

    print(tmp.nlargest(10))

    print('\n')

For topic 1 the words with the highest value are:
wa             4.113348
told           1.155619
charge         0.885329
called         0.879305
did            0.766151
chase          0.763776
said           0.718138
claim          0.649698
wa told        0.594599
transaction    0.563166
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
credit           2.755414
card             2.425318
credit card      1.496190
chase            1.028794
report           0.762285
credit report    0.591013
inquiry          0.523730
charge           0.412636
chase credit     0.396305
account          0.321179
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
chase           2.361499
loan            1.468328
mortgage        1.105126
bank            0.752978
home            0.642239
chase bank      0.616075
ha              0.606456
modification    0.491960
property        0.425811
letter          0.406402
Name: 2, dtype: float64


For topic 4 the words 

In [20]:
lb = []

for i in range(len(df)):
    lab = nmf_score.iloc[i].idxmax(axis=0)
    lb.append(lab)
    #df['Label'][i] = nmf_score.iloc[i].idxmax(axis=0)

In [22]:
df['Label'] = pd.DataFrame(lb)

C:\Users\PRATYA~1\AppData\Local\Temp/ipykernel_30404/2554644604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = pd.DataFrame(lb)


In [23]:
df

,id,issue,sub_product,complaint,sub_issue,Label
0,3211475,Attempts to collect debt not owed,Credit card debt,,Debt is not yours,0
1,3229299,Written notification about debt,Credit card debt,"[good, morning, my, name, is, and, appreciate,...",Didn't receive enough information to verify debt,2
2,3199379,"Other features, terms, or problems",General-purpose credit card or charge card,"[upgraded, my, card, in, //, and, wa, told, by...",Problem with rewards from credit card,0
3,2673060,Trouble during payment process,Conventional home mortgage,,None,0
4,3203545,Fees or interest,General-purpose credit card or charge card,,Charged too much interest,0
...,...,...,...,...,...,...
78308,3105617,Closing an account,Checking account,,Funds not received from closed account,0
78309,3091984,"Other features, terms, or problems",General-purpose credit card or charge card,"[on, wednesday, //, called, chas, my, visa, cr...",Other problem,0
78310,3133355,Problem with a lender or other company chargin...,Checking account,"[am, not, familiar, with, pay, and, did, not, ...",Transaction was not authorized,2
78311,3110963,"Other features, terms, or problems",General-purpose credit card or charge card,"[have, had, flawless, credit, for, yr, 've, ha...",Problem with balance transfer,4


In [24]:
X = df.complaint
y = df.Label

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state= 7)

In [27]:
vec = TfidfVectorizer()
vec.fit(x_train.apply(lambda x: ' '.join(x)))

TfidfVectorizer()

In [28]:
x_train = vec.transform(x_train.apply(lambda x: ' '.join(x)))
x_test = vec.transform(x_test.apply(lambda x: ' '.join(x)))

In [29]:
multi_nb = MultinomialNB()

multi_nb.fit(x_train,y_train)
y_pred = multi_nb.predict(x_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90     12442
           1       0.90      0.17      0.28      1011
           2       0.93      0.16      0.28       850
           3       1.00      0.00      0.00       877
           4       0.00      0.00      0.00       483

    accuracy                           0.81     15663
   macro avg       0.73      0.27      0.29     15663
weighted avg       0.81      0.81      0.75     15663



C:\Users\pratyaksh_agarwal1\Anaconda33\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pratyaksh_agarwal1\Anaconda33\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pratyaksh_agarwal1\Anaconda33\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()

In [31]:
sgd.fit(x_train,y_train)
y_pred1 = sgd.predict(x_test)

print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     12442
           1       0.92      0.96      0.94      1011
           2       0.94      0.83      0.88       850
           3       0.89      0.96      0.92       877
           4       0.93      0.86      0.90       483

    accuracy                           0.97     15663
   macro avg       0.93      0.92      0.93     15663
weighted avg       0.97      0.97      0.97     15663

